In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Masking, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
df = pd.read_csv(r"C:\Users\KAVISH\Downloads\FidelFolio_df3.csv")

In [4]:
for col in df.columns:
    if col not in ['Company', 'Year']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [6]:
df['target_lag1'] = df.groupby('Company')['Target 3'].shift(1)
df['Target2_lag1'] = df['target_lag1'].fillna(-999)

In [11]:
feature_cols = [col for col in df.columns if col not in ['Company', 'Year', 'Target 3']]

In [12]:
df[feature_cols] = df[feature_cols].fillna(-999)

In [13]:
SEQ_LEN = 3  # Increased sequence length
MASK_VALUE = -999.
BATCH_SIZE = 64  # Adjusted batch size
EPOCHS = 30  # Increased epochs with early stopping
INIT_LR = 0.0005  # Lower initial learning rate

In [16]:
X_train = df[[col for col in feature_cols if col in df.columns]].values

y_train = df['Target 3'].values

In [18]:

train_df = df[df['Year'] < 2022].copy()
test_df = df[df['Year'] > 2021].copy()


#train_df = train_df.dropna(subset=[' Target 1 '])

# Fill NaNs in features (you can also choose to drop or impute)
X_columns = [col for col in train_df.columns if col not in ['Company', 'Year', ' Target 1 ',' Target 2 ', ' Target 3 ']]
train_df[X_columns] = train_df[X_columns].fillna(-999)
test_df[X_columns] = test_df[X_columns].fillna(-999)


X_train = train_df[X_columns]
y_train = train_df['Target 3'].values

X_test = test_df[X_columns]

# Build the MLP model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Regression output
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Predict for 2024
predictions_2024 = model.predict(X_test)

# Add predictions to test_df
test_df['predicted_target3'] = predictions_2024

# Save the predictions
test_df[['Company', 'Year', 'predicted_target3']].to_csv("2024_predictions.csv", index=False)

Epoch 1/100
560/560 [==============================] - 3s 3ms/step - loss: 62366.1133 - mae: 126.5677 - val_loss: 23139.1973 - val_mae: 76.5808
Epoch 2/100
560/560 [==============================] - 2s 3ms/step - loss: 11438.0557 - mae: 59.1435 - val_loss: 34262.6094 - val_mae: 117.6678
Epoch 3/100
560/560 [==============================] - 2s 3ms/step - loss: 9231.6270 - mae: 51.9699 - val_loss: 40194.9922 - val_mae: 116.9867
Epoch 4/100
560/560 [==============================] - 2s 3ms/step - loss: 13473.2041 - mae: 63.6747 - val_loss: 8921.9697 - val_mae: 55.7522
Epoch 5/100
560/560 [==============================] - 2s 3ms/step - loss: 13208.2646 - mae: 51.6576 - val_loss: 419970.9688 - val_mae: 69.7727
Epoch 6/100
560/560 [==============================] - 2s 3ms/step - loss: 20461.2637 - mae: 60.9707 - val_loss: 5521.5669 - val_mae: 24.8253
Epoch 7/100
560/560 [==============================] - 2s 3ms/step - loss: 5860.8042 - mae: 39.0972 - val_loss: 14331.4277 - val_mae: 37.8262

In [22]:
test_df.to_csv('C:/Users/KAVISH/Downloads/test_data_processed3.csv', index=False)